<a href="https://colab.research.google.com/github/indranildchandra/Abstractive-Text-Summarizer/blob/master/src/Summarizer_Engine_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summurization using seq2seq lib

### Intro
This is a modification to https://github.com/dongjun-Lee/text-summarization-tensorflow 
I am builging it in a notebook envronment to be able to easily integrate with colab


## Helpers (Link Google Drive to Colab Notebook, Upload Dataset directly, Utility methods)

### Link Google Drive to Colab Notebook

In [1]:
%%time

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/16812012/+files/google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
--2019-06-14 04:56:57--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/16812012/+files/google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/424309408/google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb [following]
--2019-06-14 04:57:02--  https://launchpadlibrarian.net/424309408/google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.229, 91.189.89.228, 2001:67c:1560:8003::8008, ...
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.229|:443... connected.
HTTP request sent, awaiting resp

In [2]:
!ls -lrt

total 1220
-rw-r--r-- 1 root root 1236332 May 18 15:04 google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb
drwxr-xr-x 1 root root    4096 May 31 16:17 sample_data
-rw-r--r-- 1 root root    2557 Jun 14 04:57 adc.json
drwxr-xr-x 2 root root    4096 Jun 14 04:58 drive


### Upload Dataset directly

In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# #upload summarization_dataset.zip file for training

In [0]:
# !unzip summarization_dataset.zip

In [0]:
# !ls -lahrt

### Utilities

https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/utils.py

In [6]:
%%time

!pip install gensim
!pip install wget
  
import nltk
nltk.download('punkt')

  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 823 ms, sys: 153 ms, total: 976 ms
Wall time: 10.6 s


In [7]:
%%time

!wget https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
!unzip glove.42B.300d.zip
!ls -lrt

--2019-06-14 04:58:36--  https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip [following]
--2019-06-14 04:58:37--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877802108 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  14.0MB/s    in 2m 21s  

2019-06-14 05:00:58 (12.7 MB/s) - ‘glove.42B.300d.zip’ saved [1877802108/1877802108]

Archive:  glove.42B.300d.zip
  inflating: glove.42B.

In [8]:
!head -n 20 glove.42B.300d.txt

, 0.18378 -0.12123 -0.11987 0.015227 -0.19121 -0.066074 -2.9876 0.80795 0.067338 -0.13184 -0.5274 0.44521 0.12982 -0.21823 -0.4508 -0.22478 -0.30766 -0.11137 -0.162 -0.21294 -0.46022 -0.086593 -0.24902 0.46729 -0.6023 -0.44972 0.43946 0.014738 0.27498 -0.078421 0.36009 0.12172 0.4298 -0.055345 0.4495 -0.74444 -0.26702 0.16431 -0.19335 0.13468 0.2887 0.23924 -0.23579 -0.28972 0.20149 0.048135 -0.18322 -0.15492 -0.19255 0.40271 0.16051 0.17721 0.32557 0.011625 -0.42572 0.34205 -0.45865 -0.2486 0.034128 0.03306 -0.057065 0.18136 -0.43638 0.0005709 -0.11935 -0.2195 0.16429 -0.18119 -0.19145 -0.081672 -0.2962 0.25803 0.073848 0.54213 -0.15405 -0.49256 0.091719 0.13329 -0.05253 -0.20518 0.34576 -1.0449 0.072779 -0.0003453 -0.16926 0.051019 -0.14753 0.23848 -0.40749 -0.58278 -0.48695 0.25863 -0.20531 -0.4775 0.40645 -0.038512 -2.403 -0.12421 0.63149 0.089419 0.08557 -0.20757 -0.1617 -0.29506 -0.13948 0.14202 -0.30138 -0.15806 0.52984 0.24229 0.075169 0.13792 0.90416 -0.23647 0.027788 0.099915

In [0]:
## Example to directly generate GloVe vector embeddings from text file
# !python -m gensim.scripts.glove2word2vec --input glove.42B.300d.txt --output model_glove_300.vec
# !ls -lrt

In [0]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

default_path = "drive/Colab Notebooks/summarization_dataset/"

train_article_path = default_path + "train/train.article.txt"
train_title_path   = default_path + "train/train.title.txt"
valid_article_path = default_path + "train/valid.article.filter.txt"
valid_title_path   = default_path + "train/valid.title.filter.txt"

#valid_article_path = default_path + "DUC2003/input.txt"
#valid_title_path   = default_path + "DUC2003/task1_ref0.txt"

def clean_str(sentence):
    sentence = re.sub("[#.]+", "#", sentence)
    return sentence


def get_text_list(data_path, toy):
    with open (data_path, "r", encoding="utf-8") as f:
        if not toy:
            return [clean_str(x.strip()) for x in f.readlines()][:200000]
        else:
            return [clean_str(x.strip()) for x in f.readlines()][:50]


def build_dict(step, toy=False):
    if step == "train":
        train_article_list = get_text_list(train_article_path, toy)
        train_title_list = get_text_list(train_title_path, toy)

        words = list()
        for sentence in train_article_list + train_title_list:
            for word in word_tokenize(sentence):
                words.append(word)

        word_counter = collections.Counter(words).most_common()
        word_dict = dict()
        word_dict["<padding>"] = 0
        word_dict["<unk>"] = 1
        word_dict["<s>"] = 2
        word_dict["</s>"] = 3
        for word, _ in word_counter:
            word_dict[word] = len(word_dict)

        with open("word_dict.pickle", "wb") as f:
            pickle.dump(word_dict, f)

    elif step == "valid":
        with open("word_dict.pickle", "rb") as f:
            word_dict = pickle.load(f)

    reversed_dict = dict(zip(word_dict.values(), word_dict.keys()))

    article_max_len = 50
    summary_max_len = 15

    return word_dict, reversed_dict, article_max_len, summary_max_len


def build_dataset(step, word_dict, article_max_len, summary_max_len, toy=False):
    if step == "train":
        article_list = get_text_list(train_article_path, toy)
        title_list = get_text_list(train_title_path, toy)
    elif step == "valid":
        article_list = get_text_list(valid_article_path, toy)
    else:
        raise NotImplementedError

    x = [word_tokenize(d) for d in article_list]
    x = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in x]
    x = [d[:article_max_len] for d in x]
    x = [d + (article_max_len - len(d)) * [word_dict["<padding>"]] for d in x]
    
    if step == "valid":
        return x
    else:        
        y = [word_tokenize(d) for d in title_list]
        y = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in y]
        y = [d[:(summary_max_len - 1)] for d in y]
        return x, y


def batch_iter(inputs, outputs, batch_size, num_epochs):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]


def get_init_embedding(reversed_dict, embedding_size):
    glove_file = "glove.42B.300d.txt"
    word2vec_file = get_tmpfile("word2vec_format.vec")
    glove2word2vec(glove_file, word2vec_file)
    print("Loading Glove vectors...")
    word_vectors = KeyedVectors.load_word2vec_format(word2vec_file)

#     with open("model_glove_300.pkl", 'rb') as handle:
#         word_vectors = pickle.load(handle)
        
    word_vec_list = list()
    for _, word in sorted(reversed_dict.items()):
        try:
            word_vec = word_vectors.word_vec(word)
        except KeyError:
            word_vec = np.zeros([embedding_size], dtype=np.float32)

        word_vec_list.append(word_vec)

    # Assign random vector to <s>, </s> token
    word_vec_list[2] = np.random.normal(0, 1, embedding_size)
    word_vec_list[3] = np.random.normal(0, 1, embedding_size)

    return np.array(word_vec_list)

## Prepare Data


https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/prep_data.py

1.   Word Embedding : Used [Glove pre-trained vectors](https://nlp.stanford.edu/projects/glove/ ) to initialize word embedding.  
2.   Dataset :  Dataset is available at [harvardnlp/sent-summary](https://github.com/harvardnlp/sent-summary). Locate the summary.tar.gz file in project root directory.   



In [0]:
import wget
import os
import tarfile
import gzip
import zipfile
import argparse


#parser = argparse.ArgumentParser()
#parser.add_argument("--glove", action="store_true")
#args = parser.parse_args()

# Extract data file
#with tarfile.open(default_path + "train/summary.tar.gz", "r:gz") as tar:
#    tar.extractall()

# with gzip.open(default_path + "train/train.article.txt.gz", "rb") as gz:
#     with open(default_path + "train/train.article.txt", "wb") as out:
#         out.write(gz.read())

# with gzip.open(default_path + "train/train.title.txt.gz", "rb") as gz:
#     with open(default_path + "train/train.title.txt", "wb") as out:
#         out.write(gz.read())

        
# if args.glove:
#    glove_dir = "glove"
#    glove_url = "https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip"

#    if not os.path.exists(glove_dir):
#        os.mkdir(glove_dir)

#    # Download glove vector
#    wget.download(glove_url, out=glove_dir)

#    # Extract glove file
#    with zipfile.ZipFile(os.path.join("glove", "glove.42B.300d.zip"), "r") as z:
#        z.extractall(glove_dir)

##Model:



Encoder-Decoder model with attention mechanism.
https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/model.py

1.   Encoder : Used LSTM cell with stack_bidirectional_dynamic_rnn.
2.   Decoder : Used LSTM BasicDecoder for training, and BeamSearchDecoder for inference.
3.   Attention Mechanism : Used BahdanauAttention with weight normalization.

In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn

class Model(object):
    def __init__(self, reversed_dict, article_max_len, summary_max_len, args, forward_only=False):
        self.vocabulary_size = len(reversed_dict)
        self.embedding_size = args.embedding_size
        self.num_hidden = args.num_hidden
        self.num_layers = args.num_layers
        self.learning_rate = args.learning_rate
        self.beam_width = args.beam_width
        if not forward_only:
            self.keep_prob = args.keep_prob
        else:
            self.keep_prob = 1.0
        self.cell = tf.nn.rnn_cell.BasicLSTMCell
        with tf.variable_scope("decoder/projection"):
            self.projection_layer = tf.layers.Dense(self.vocabulary_size, use_bias=False)

        self.batch_size = tf.placeholder(tf.int32, (), name="batch_size")
        self.X = tf.placeholder(tf.int32, [None, article_max_len])
        self.X_len = tf.placeholder(tf.int32, [None])
        self.decoder_input = tf.placeholder(tf.int32, [None, summary_max_len])
        self.decoder_len = tf.placeholder(tf.int32, [None])
        self.decoder_target = tf.placeholder(tf.int32, [None, summary_max_len])
        self.global_step = tf.Variable(0, trainable=False)

        with tf.name_scope("embedding"):
            if not forward_only and args.glove:
                init_embeddings = tf.constant(get_init_embedding(reversed_dict, self.embedding_size), dtype=tf.float32)
            else:
                init_embeddings = tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0)
            self.embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
            self.encoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.X), perm=[1, 0, 2])
            self.decoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.decoder_input), perm=[1, 0, 2])

        with tf.name_scope("encoder"):
            fw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            bw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            fw_cells = [rnn.DropoutWrapper(cell) for cell in fw_cells]
            bw_cells = [rnn.DropoutWrapper(cell) for cell in bw_cells]

            encoder_outputs, encoder_state_fw, encoder_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                fw_cells, bw_cells, self.encoder_emb_inp,
                sequence_length=self.X_len, time_major=True, dtype=tf.float32)
            self.encoder_output = tf.concat(encoder_outputs, 2)
            encoder_state_c = tf.concat((encoder_state_fw[0].c, encoder_state_bw[0].c), 1)
            encoder_state_h = tf.concat((encoder_state_fw[0].h, encoder_state_bw[0].h), 1)
            self.encoder_state = rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)

        with tf.name_scope("decoder"), tf.variable_scope("decoder") as decoder_scope:
            decoder_cell = self.cell(self.num_hidden * 2)

            if not forward_only:
                attention_states = tf.transpose(self.encoder_output, [1, 0, 2])
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, attention_states, memory_sequence_length=self.X_len, normalize=True)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size)
                initial_state = initial_state.clone(cell_state=self.encoder_state)
                helper = tf.contrib.seq2seq.TrainingHelper(self.decoder_emb_inp, self.decoder_len, time_major=True)
                decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True, scope=decoder_scope)
                self.decoder_output = outputs.rnn_output
                self.logits = tf.transpose(
                    self.projection_layer(self.decoder_output), perm=[1, 0, 2])
                self.logits_reshape = tf.concat(
                    [self.logits, tf.zeros([self.batch_size, summary_max_len - tf.shape(self.logits)[1], self.vocabulary_size])], axis=1)
            else:
                tiled_encoder_output = tf.contrib.seq2seq.tile_batch(
                    tf.transpose(self.encoder_output, perm=[1, 0, 2]), multiplier=self.beam_width)
                tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(self.encoder_state, multiplier=self.beam_width)
                tiled_seq_len = tf.contrib.seq2seq.tile_batch(self.X_len, multiplier=self.beam_width)
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, tiled_encoder_output, memory_sequence_length=tiled_seq_len, normalize=True)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size * self.beam_width)
                initial_state = initial_state.clone(cell_state=tiled_encoder_final_state)
                decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                    cell=decoder_cell,
                    embedding=self.embeddings,
                    start_tokens=tf.fill([self.batch_size], tf.constant(2)),
                    end_token=tf.constant(3),
                    initial_state=initial_state,
                    beam_width=self.beam_width,
                    output_layer=self.projection_layer
                )
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder, output_time_major=True, maximum_iterations=summary_max_len, scope=decoder_scope)
                self.prediction = tf.transpose(outputs.predicted_ids, perm=[1, 2, 0])

        with tf.name_scope("loss"):
            if not forward_only:
                crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=self.logits_reshape, labels=self.decoder_target)
                weights = tf.sequence_mask(self.decoder_len, summary_max_len, dtype=tf.float32)
                self.loss = tf.reduce_sum(crossent * weights / tf.to_float(self.batch_size))

                params = tf.trainable_variables()
                gradients = tf.gradients(self.loss, params)
                clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
                optimizer = tf.train.AdamOptimizer(self.learning_rate)
                self.update = optimizer.apply_gradients(zip(clipped_gradients, params), global_step=self.global_step)

## Train

https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/train.py

Use train/train.article.txt and train/train.title.txt for training data. 

In [14]:
%%time

import time
start = time.perf_counter()
import tensorflow as tf
import argparse
import pickle
import os
#from model import Model
#from utils import build_dict, build_dataset, batch_iter

# Uncomment next 2 lines to suppress error and Tensorflow info verbosity. Or change logging levels
# tf.logging.set_verbosity(tf.logging.FATAL)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#def add_arguments(parser):
#    parser.add_argument("--num_hidden", type=int, default=150, help="Network size.")
#    parser.add_argument("--num_layers", type=int, default=2, help="Network depth.")
#    parser.add_argument("--beam_width", type=int, default=10, help="Beam width for beam search decoder.")
#    parser.add_argument("--glove", action="store_true", help="Use glove as initial word embedding.")
#    parser.add_argument("--embedding_size", type=int, default=300, help="Word embedding size.")
#
#    parser.add_argument("--learning_rate", type=float, default=1e-3, help="Learning rate.")
#    parser.add_argument("--batch_size", type=int, default=64, help="Batch size.")
#    parser.add_argument("--num_epochs", type=int, default=10, help="Number of epochs.")
#    parser.add_argument("--keep_prob", type=float, default=0.8, help="Dropout keep prob.")
#
#    parser.add_argument("--toy", action="store_true", help="Use only 50K samples of data")
#
#    parser.add_argument("--with_model", action="store_true", help="Continue from previously saved model")

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove="store_true"
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size=64
args.num_epochs=10
args.keep_prob = 0.8

args.toy=False #"store_true"

args.with_model="store_true"


#parser = argparse.ArgumentParser()
#add_arguments(parser)
#args = parser.parse_args()
#with open("args.pickle", "wb") as f:
#    pickle.dump(args, f)

if not os.path.exists("saved_model"):
    os.mkdir("saved_model")
else:
    #if args.with_model:
        old_model_checkpoint_path = open('saved_model/checkpoint', 'r')
        old_model_checkpoint_path = "".join(["saved_model/",old_model_checkpoint_path.read().splitlines()[0].split('"')[1] ])


print("Building dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train", args.toy)
print("Loading training dataset...")
train_x, train_y = build_dataset("train", word_dict, article_max_len, summary_max_len, args.toy)

tf.reset_default_graph()

with tf.Session() as sess:
    model = Model(reversed_dict, article_max_len, summary_max_len, args)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    if 'old_model_checkpoint_path' in globals():
        print("Continuing from previous trained model:" , old_model_checkpoint_path , "...")
        saver.restore(sess, old_model_checkpoint_path )

    batches = batch_iter(train_x, train_y, args.batch_size, args.num_epochs)
    num_batches_per_epoch = (len(train_x) - 1) // args.batch_size + 1

    print("\nIteration starts.")
    print("Number of batches per epoch :", num_batches_per_epoch)
    for batch_x, batch_y in batches:
        batch_x_len = list(map(lambda x: len([y for y in x if y != 0]), batch_x))
        batch_decoder_input = list(map(lambda x: [word_dict["<s>"]] + list(x), batch_y))
        batch_decoder_len = list(map(lambda x: len([y for y in x if y != 0]), batch_decoder_input))
        batch_decoder_output = list(map(lambda x: list(x) + [word_dict["</s>"]], batch_y))

        batch_decoder_input = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_input))
        batch_decoder_output = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_output))

        train_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
            model.decoder_input: batch_decoder_input,
            model.decoder_len: batch_decoder_len,
            model.decoder_target: batch_decoder_output
        }

        _, step, loss = sess.run([model.update, model.global_step, model.loss], feed_dict=train_feed_dict)

        if step % 1000 == 0:
            print("step {0}: loss = {1}".format(step, loss))

        if step % num_batches_per_epoch == 0:
            hours, rem = divmod(time.perf_counter() - start, 3600)
            minutes, seconds = divmod(rem, 60)
            saver.save(sess, "saved_model/model.ckpt", global_step=step)
            print(" Epoch {0}: Model is saved.".format(step // num_batches_per_epoch),
            "Elapsed: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds) , "\n")

Building dictionary...
Loading training dataset...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Loading Glove vectors...


W0614 05:16:01.223193 139630703044480 deprecation.py:323] From <ipython-input-13-3e3c149b757b>:40: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0614 05:16:01.239972 139630703044480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/rnn/python/ops/rnn.py:239: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
W0614 05:16:01.243323 139630703044480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Pl


Iteration starts.
Number of batches per epoch : 3125
step 1000: loss = 48.56650161743164
step 2000: loss = 37.45339584350586
step 3000: loss = 35.38955307006836
 Epoch 1: Model is saved. Elapsed: 00:25:17.55 

step 4000: loss = 33.68756866455078
step 5000: loss = 35.29840087890625
step 6000: loss = 27.300079345703125
 Epoch 2: Model is saved. Elapsed: 00:38:01.38 

step 7000: loss = 30.049083709716797
step 8000: loss = 22.10747528076172
step 9000: loss = 21.429994583129883
 Epoch 3: Model is saved. Elapsed: 00:50:45.61 

step 10000: loss = 19.815256118774414
step 11000: loss = 17.53327751159668
step 12000: loss = 13.953457832336426
 Epoch 4: Model is saved. Elapsed: 01:03:29.58 

step 13000: loss = 19.24254608154297
step 14000: loss = 12.913619041442871
step 15000: loss = 17.103111267089844
 Epoch 5: Model is saved. Elapsed: 01:16:06.93 

step 16000: loss = 16.89795684814453
step 17000: loss = 17.54222297668457
step 18000: loss = 15.565654754638672


W0614 06:32:16.249577 139630703044480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


 Epoch 6: Model is saved. Elapsed: 01:28:41.07 

step 19000: loss = 15.311954498291016
step 20000: loss = 13.111289978027344
step 21000: loss = 10.104683876037598
 Epoch 7: Model is saved. Elapsed: 01:41:11.20 

step 22000: loss = 9.558016777038574
step 23000: loss = 10.83720588684082
step 24000: loss = 8.748115539550781
step 25000: loss = 14.962991714477539
 Epoch 8: Model is saved. Elapsed: 01:53:37.65 

step 26000: loss = 15.608682632446289
step 27000: loss = 9.522758483886719
step 28000: loss = 11.305787086486816
 Epoch 9: Model is saved. Elapsed: 02:06:03.44 

step 29000: loss = 10.565549850463867
step 30000: loss = 16.50485610961914
step 31000: loss = 8.069765090942383
 Epoch 10: Model is saved. Elapsed: 02:18:29.85 

CPU times: user 2h 50min 24s, sys: 18min 53s, total: 3h 9min 17s
Wall time: 2h 18min 33s


## Test

https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/test.py

Generate summary of each article in sumdata/train/valid.article.filter.txt by

In [15]:
%%time

import tensorflow as tf
import pickle
#from model import Model
#from utils import build_dict, build_dataset, batch_iter


#with open("args.pickle", "rb") as f:
#    args = pickle.load(f)

tf.reset_default_graph()

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove="store_true"
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size=64
args.num_epochs=10
args.keep_prob = 0.8

args.toy=True

args.with_model="store_true"



print("Loading dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
print("Loading validation dataset...")
valid_x = build_dataset("valid", word_dict, article_max_len, summary_max_len, args.toy)
valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]
print("Loading article and reference...")
article = get_text_list(valid_article_path, args.toy)
reference = get_text_list(valid_title_path, args.toy)

with tf.Session() as sess:
    print("Loading saved model...")
    model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True)
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state("saved_model/")
    saver.restore(sess, ckpt.model_checkpoint_path)

    batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)

    print("Writing summaries to 'result.txt'...")
    for batch_x, _ in batches:
        batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]

        valid_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
        }

        prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
        prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
        summary_array = []
        with open("result.txt", "a") as f:
            for line in prediction_output:
                summary = list()
                for word in line:
                    if word == "</s>":
                        break
                    if word not in summary:
                        summary.append(word)
                summary_array.append(" ".join(summary))
                #print(" ".join(summary), file=f)

    print('Summaries have been generated')

Loading dictionary...
Loading validation dataset...
Loading article and reference...
Loading saved model...


W0614 07:22:19.956315 139630703044480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/seq2seq/python/ops/beam_search_decoder.py:985: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0614 07:22:20.185108 139630703044480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Writing summaries to 'result.txt'...
Summaries have been generated
CPU times: user 4.72 s, sys: 173 ms, total: 4.89 s
Wall time: 14.2 s


## Evaluate & write output

for comparing (good resources)

1.  [thunlp]( https://github.com/thunlp/TensorFlow-Summarization)     works with duc2003
2.  [textsum](https://github.com/tensorflow/models/tree/master/research/textsum )   



In [16]:
!pip install sumeval
!python -m spacy download en

     |████████████████████████████████| 51kB 2.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/85/9b/50/fda7087af57d2efde31d4a85cecda98e683ed58eee07fbff2b
  Stored in directory: /root/.cache/pip/wheels/f5/21/da/3e3dea18eb53af3b56d99b4b6ae50c91c89c8a7f422d676ee1
Successfully built sumeval sacrebleu
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
#https://github.com/chakki-works/sumeval
#https://github.com/Tian312/awesome-text-summarization

from sumeval.metrics.rouge import RougeCalculator
from sumeval.metrics.bleu import BLEUCalculator

def eval_rouges(refrence_summary,model_summary):
    #refrence_summary = "tokyo shares close up #.## percent"
    #model_summary = "tokyo stocks close up # percent to fresh record high"

    rouge = RougeCalculator(stopwords=True, lang="en")

    rouge_1 = rouge.rouge_n(
                summary=model_summary,
                references=refrence_summary,
                n=1)

    rouge_2 = rouge.rouge_n(
                summary=model_summary,
                references=[refrence_summary],
                n=2)
    
    rouge_l = rouge.rouge_l(
                summary=model_summary,
                references=[refrence_summary])
    
    # You need spaCy to calculate ROUGE-BE
    
    rouge_be = rouge.rouge_be(
                summary=model_summary,
                references=[refrence_summary])

    bleu = BLEUCalculator()
    bleu_score = bleu.bleu( summary=model_summary,
                        references=[refrence_summary])

    #print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-L: {}, ROUGE-BE: {}".format(
    #    rouge_1, rouge_2, rouge_l, rouge_be
    #).replace(", ", "\n"))
    
    return rouge_1, rouge_2,rouge_l,rouge_be,bleu_score
  
#rouge_1, rouge_2,rouge_l,rouge_be = eval_rouges( "tokyo shares close up #.## percent",
#                                                "tokyo stocks close up # percent to fresh record high")
#
#print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-L: {}, ROUGE-BE: {}".format(
#        rouge_1, rouge_2, rouge_l, rouge_be
#    ).replace(", ", "\n"))

In [18]:
#https://pymotw.com/2/xml/etree/ElementTree/create.html

bleu_arr = []
rouge_1_arr  = []
rouge_2_arr  = []
rouge_L_arr  = []
rouge_be_arr = []

from xml.etree import ElementTree
from xml.dom import minidom
from functools import reduce

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")
  
from xml.etree.ElementTree import Element, SubElement, Comment

top = Element('ZakSum')

comment = Comment('Generated by Amr Zaki')
top.append(comment)

i=0
for summ in summary_array:
  example = SubElement(top, 'example')
  article_element   = SubElement(example, 'article')
  article_element.text = article[i]
  
  reference_element = SubElement(example, 'reference')
  reference_element.text = reference[i]
  
  summary_element   = SubElement(example, 'summary')
  summary_element.text = summ

  rouge_1, rouge_2,rouge_L,rouge_be,bleu_score = eval_rouges(reference[i],summ )
  
  eval_element = SubElement(example, 'eval')
  bleu_score_element = SubElement(eval_element,'BLEU', {'score':str(bleu_score)})
  ROUGE_1_element  = SubElement(eval_element, 'ROUGE_1' , {'score':str(rouge_1)})
  ROUGE_2_element  = SubElement(eval_element, 'ROUGE_2' , {'score':str(rouge_2)})
  ROUGE_L_element  = SubElement(eval_element, 'ROUGE_l' , {'score':str(rouge_L)})
  ROUGE_be_element  = SubElement(eval_element,'ROUGE_be', {'score':str(rouge_be)})
  
  bleu_arr.append(bleu_score) 
  rouge_1_arr.append(rouge_1) 
  rouge_2_arr.append(rouge_2) 
  rouge_L_arr.append(rouge_L) 
  rouge_be_arr.append(rouge_be) 

  i+=1

top.set('bleu', str(reduce(lambda x, y: x + y,  bleu_arr) / len(bleu_arr)))
top.set('rouge_1', str(reduce(lambda x, y: x + y,  rouge_1_arr) / len(rouge_1_arr)))
top.set('rouge_2', str(reduce(lambda x, y: x + y,  rouge_2_arr) / len(rouge_2_arr)))
top.set('rouge_L', str(reduce(lambda x, y: x + y,  rouge_L_arr) / len(rouge_L_arr)))
top.set('rouge_be', str(reduce(lambda x, y: x + y, rouge_be_arr) / len(rouge_be_arr)))

with open("result_valid.xml", "w+") as f:
  print(prettify(top), file=f)

a.kwan=(nsubj)=>withdraws
<BasicElement: kwan-[nsubj]->withdraw>
b.olympic=(amod)=>skating
a.event=(dobj)=>withdraws
<BasicElement: event-[dobj]->withdraw>
a.injury=(nsubj)=>leaves
<BasicElement: injury-[nsubj]->leave>
a.kwan=(dobj)=>leaves
<BasicElement: kwan-[dobj]->leave>
b.olympic=(amod)=>hopes
a.leaders=(nsubj)=>lash
<BasicElement: leaders-[nsubj]->lash>
b.illegal=(amod)=>immigrants
a.attacks=(nsubj)=>tough
<BasicElement: attacks-[nsubj]->tough>
a.law=(dobj)=>tough
<BasicElement: law-[dobj]->tough>
a.gm=(mark)=>fall
<BasicElement: gm-[mark]->fall>
a.sales=(nsubj)=>fall
<BasicElement: sales-[nsubj]->fall>
a.percent=(dobj)=>fall
<BasicElement: percent-[dobj]->fall>
a.sales=(nsubj)=>fall
<BasicElement: sales-[nsubj]->fall>
a.percent=(dobj)=>fall
<BasicElement: percent-[dobj]->fall>
a.thousands=(nummod)=>protest
<BasicElement: thousands-[nummod]->protest>
a.thousands=(nsubj)=>celebrate
<BasicElement: thousands-[nsubj]->celebrate>
a.cup=(compound)=>slalom
<BasicElement: cup-[compound]-

In [19]:
len(summary_array)

50

In [20]:
!ls -lrt

total 6743596
-rw-rw-r-- 1 root root 5025032584 Dec 22  2015 glove.42B.300d.txt
-rw-r--r-- 1 root root 1877802108 Dec 22  2015 glove.42B.300d.zip
-rw-r--r-- 1 root root    1236332 May 18 15:04 google-drive-ocamlfuse_0.7.4-0ubuntu1~ubuntu18.10.1_amd64.deb
drwxr-xr-x 1 root root       4096 May 31 16:17 sample_data
-rw-r--r-- 1 root root       2557 Jun 14 04:57 adc.json
drwxr-xr-x 2 root root       4096 Jun 14 04:58 drive
-rw-r--r-- 1 root root    1314472 Jun 14 05:05 word_dict.pickle
drwxr-xr-x 2 root root       4096 Jun 14 07:22 saved_model
-rw-r--r-- 1 root root          0 Jun 14 07:22 result.txt
-rw-r--r-- 1 root root      28292 Jun 14 07:22 result_valid.xml
